<a href="https://colab.research.google.com/github/bishuprojects1610/Dog-vs-Cat/blob/master/Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

!pip install pyDrive

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from tensorflow.keras import layers
from tensorflow import keras 
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf




Using TensorFlow backend.


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
!git clone "https://github.com/bishuprojects1610/Dog-vs-Cat"

Cloning into 'Dog-vs-Cat'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [0]:
path = '/content/drive/My Drive/content/data'
!ls '/content/drive/My Drive/content/data'

sampleSubmission.csv  test  train


In [0]:
import zipfile
with zipfile .ZipFile("/content/drive/My Drive/content/data/test1.zip","r") as zip_ref:
  zip_ref.extractall("/content/drive/My Drive/content/data/test")

In [0]:
#load data
train_data_dir = path + 'train/train'
test_data_dir = path + 'test/test'

In [0]:
IMG_HEIGHT = 50
IMG_WIDTH = 50
NUM_CHANNELS = 3

In [0]:
from threading import current_thread, Thread, Lock
from multiprocessing import Queue

In [0]:
#initialization for thread
batch_size = 500
num_train_images = 25000
num_test_images = 12500
num_train_threads = int(num_train_images/batch_size)  # 50
num_test_threads = int(num_test_images/batch_size)    # 25
lock = Lock()

In [0]:
#use queue to collect results from thread
def initialize_queue():
    queue = Queue()
    return queue

In [0]:
from os import listdir
from os.path import join, basename
from PIL import Image
import numpy as np
import pandas as pd

train_dir_path = path + "/train/train"
test_dir_path = path + "/test/test1"
train_imgs = [join(train_dir_path,f) for f in listdir(train_dir_path)]
test_imgs = [join(test_dir_path,f) for f in listdir(test_dir_path)]
print(len(train_imgs))
print(len(test_imgs))

25000
12500


In [0]:
#one-hot encoding for image file
def get_img_label(fpath):
  category = fpath.split(".")[-3]
  if category == "dog":
    return [1,0]
  elif category == "cat":
    return [0,1]

In [0]:
def get_img_array_labels(fpaths, queue):
  img_array = None
  labels = []
  for f in fpaths:
    arr = Image.open(f)
    arr = arr.resize((IMG_HEIGHT,IMG_WIDTH), Image.ANTIALIAS)
    arr = np.reshape(arr, (-1, IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS))
    if img_array is None:
      img_array=arr
    else:
      img_array = np.vstack((img_array,arr))
    labels.append(get_img_label(basename(f)))
  labels = np.array(labels)
  queue.put((img_array,labels))

In [0]:
#convert test images to np array
def get_img_array(fpaths,queue):
  img_array = None
  for f in fpaths:
    arr = Image.open(f)
    arr = arr.resize((IMG_HEIGHT,IMG_WIDTH), Image.ANTIALIAS)
    arr = np.reshape(arr, (-1, IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS))
    if img_array is None:
      img_array = arr
    else:
      img_array = np.vstack((img_array,arr))
  queue.put(img_array)


In [0]:
def dump_array(fname,arr):
  with open(fname,'wb')as f:
    pickle.dump(arr,f)
    

In [0]:
def load_pickled_array(fname , arr):
  with open(fname, 'rb') as f:
    return pickle.load(f)

In [0]:
#use threading to combine training array and lables

def get_training_data():
  threads_list = list()
  train_x = None
  train_y = []
  queue = initialize_queue()
  
  #loop over all threads to create
  for thread_index in range(num_train_threads):
    start_index = thread_index * batch_size
    end_index = (thread_index +1) * batch_size
    file_batch = train_imgs[start_index:end_index]
    thread = Thread(target = get_img_array_labels, args=(file_batch, queue))
    thread.start()
    threads_list.append(thread)
    
    print("Thread: {}, start index: {}, end index: {}".format(thread.name,start_index,end_index))
  for t in threads_list:
    t.join()
    
  while not queue.empty():
    arr, labels = queue.get()
    train_y.extend(labels)
    if train_x is None:
      train_x = arr
    else:
      train_x = np.vstack((train_x,arr))
  return train_x , train_y
    

In [0]:
def get_testing_data():
    threads_list = list()
    test_x = None
    queue = initialize_queue()
    # iterate over num of threads to create
    for thread_index in range(num_test_threads):
        start_index = thread_index * batch_size
        end_index = (thread_index + 1) * batch_size
        file_batch = train_imgs[start_index:end_index]
        thread = Thread(target =get_img_array, args=(file_batch, queue))
        thread.start()
        print("Thread: {}, start index: {}, end index: {}".format(thread.name, start_index, end_index))
        threads_list.append(thread)
    
    # join threads
    for t in threads_list:
        t.join()
        print("Thread: {} joined", t.name)
    while not queue.empty():
        arr= queue.get()
        if test_x is None:
            test_x = arr
        else:
            test_x = np.vstack((test_x, arr))
    return test_x

In [0]:
train_x, train_y = get_training_data()

Thread: Thread-57, start index: 0, end index: 500
Thread: Thread-58, start index: 500, end index: 1000
Thread: Thread-59, start index: 1000, end index: 1500
Thread: Thread-60, start index: 1500, end index: 2000
Thread: Thread-61, start index: 2000, end index: 2500
Thread: Thread-62, start index: 2500, end index: 3000
Thread: Thread-63, start index: 3000, end index: 3500
Thread: Thread-64, start index: 3500, end index: 4000
Thread: Thread-65, start index: 4000, end index: 4500
Thread: Thread-66, start index: 4500, end index: 5000
Thread: Thread-67, start index: 5000, end index: 5500
Thread: Thread-68, start index: 5500, end index: 6000
Thread: Thread-69, start index: 6000, end index: 6500
Thread: Thread-70, start index: 6500, end index: 7000
Thread: Thread-71, start index: 7000, end index: 7500
Thread: Thread-72, start index: 7500, end index: 8000
Thread: Thread-73, start index: 8000, end index: 8500
Thread: Thread-74, start index: 8500, end index: 9000
Thread: Thread-75, start index: 9

In [0]:
print(train_x.shape)
print(len(train_y))

(25000, 50, 50, 3)
25000


In [0]:
test_x =get_testing_data()
print(test_x.shape)

Thread: Thread-107, start index: 0, end index: 500
Thread: Thread-108, start index: 500, end index: 1000
Thread: Thread-109, start index: 1000, end index: 1500
Thread: Thread-110, start index: 1500, end index: 2000
Thread: Thread-111, start index: 2000, end index: 2500
Thread: Thread-112, start index: 2500, end index: 3000
Thread: Thread-113, start index: 3000, end index: 3500
Thread: Thread-114, start index: 3500, end index: 4000
Thread: Thread-115, start index: 4000, end index: 4500
Thread: Thread-116, start index: 4500, end index: 5000
Thread: Thread-117, start index: 5000, end index: 5500
Thread: Thread-118, start index: 5500, end index: 6000
Thread: Thread-119, start index: 6000, end index: 6500
Thread: Thread-120, start index: 6500, end index: 7000
Thread: Thread-121, start index: 7000, end index: 7500
Thread: Thread-122, start index: 7500, end index: 8000
Thread: Thread-123, start index: 8000, end index: 8500
Thread: Thread-124, start index: 8500, end index: 9000
Thread: Thread-

Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 397, in _send_bytes
    self._send(header)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 397, in _send_bytes
    self._send(header)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPi

In [0]:
import pickle
dump_array('train_arr.pickle',train_x)
dump_array('train_labels.pickle',train_y)

In [0]:
dump_array('test_arr.pickle',test_x)

In [0]:
print("train_x shape",train_x.shape)
print("test_x shape", test_x.shape)
# convert train_y to np. array
train_y = np.array(train_y)
print("train_y.shape", train_y.shape)

train_x shape (25000, 50, 50, 3)
test_x shape (500, 50, 50, 3)
train_y.shape (25000, 2)


In [0]:
train_x = train_x/255
test_x = test_x/255

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils, to_categorical

from sklearn.model_selection import train_test_split

In [0]:
model = Sequential()

In [0]:
model.add(Conv2D(16, (3,3), input_shape=(50,50,3))) 
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))

In [0]:
model.add(MaxPooling2D(pool_size=(2,2),strides=2)) 

In [0]:
model.add(Conv2D(16, (3,3)), ) 
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))

In [0]:
model.add(MaxPooling2D(pool_size=(2,2),strides=2))

In [0]:
model.add(Conv2D(32, (3,3)))                     
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))

In [0]:
model.add(MaxPooling2D(pool_size=(2,2),strides=2))

In [0]:
model.add(Conv2D(32, (3,3))) 
model.add(BatchNormalization(axis=3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))

In [0]:
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

ValueError: ignored

In [0]:
print(onehot_encoded_arr[:2])

NameError: ignored

In [0]:
predictions = model.predict(test_x, batch_size=32, verbose=1)

500/500 [==============================] - 1s 2ms/step


In [0]:
model.summary()

ValueError: ignored

In [0]:
!git init

Initialized empty Git repository in /content/.git/
